# Script de Geração do Template de Resultados do Focus

Execução do Código:

* Entra no site do BC e baixa o documento do focus da segunda respectiva;
* Lê o PDF e separa os dados de interesse;
* Por fim os dados são adicionados ao template.

Pontos de Atenção:

* Modificar "base_path" e "download_directory" de acordo com seu PC;
* Pode ser que ao ler o PDF os números venham com espaços enter os dígitos ou algo similar. O código não consegue lidar com esse tipo de erro, neste caso sugere-se alterar o template original.

Link template original: https://www.canva.com/design/DAFo6BHnFLo/wSjpGwHscBFz2tyPrBT44w/edit

# 0. Import Libraries

In [ ]:
import os
import locale
import PyPDF2
import requests
import pandas as pd

from io import BytesIO
from bcb import Expectativas
from datetime import datetime, timedelta
from PIL import Image, ImageDraw, ImageFont

## Warnings Off
import warnings
warnings.filterwarnings("ignore")

In [ ]:
locale.setlocale(locale.LC_ALL, 'pt_BR')

## Today
today = datetime.today().strftime('%Y%m%d')

curr_year = datetime.today().year
next_year = curr_year + 1
print(today)

# 1. Collecting Focus Data

In [ ]:
def get_last_friday():
    today = datetime.today()
    today_weekday = today.weekday()  # Monday is 0, Sunday is 6
    
    ## Calculating the number of days to subtract to get to the last Friday (weekday 4)
    days_difference = today_weekday - 4 if today_weekday >= 4 else today_weekday + 3

    ## Subtracting the calculated days to get the date of the last Friday
    last_friday = today - timedelta(days=days_difference)

    return last_friday

In [ ]:
## Setting the download path
download_directory = os.path.join(os.path.expanduser('~'), 'Documents')

## Setting the URL of the PDF
last_friday = get_last_friday().strftime('%Y%m%d')
pdf_url = f'https://www.bcb.gov.br/content/focus/focus/R{last_friday}.pdf'
response = requests.get(pdf_url)

## Checking if successful
if response.status_code == 200:
    file_name = os.path.basename(pdf_url)
    download_path = os.path.join(download_directory, file_name)
    
    ## Saving the PDF content
    with open(download_path, "wb") as f:
        f.write(response.content)
    
    print(f"Downloaded: {download_path}")
    
else:
    print("Failed to download the PDF.")

In [ ]:
## Reading the PDF file
with open(download_path, "rb") as pdf_file:
    ## PDF reader
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    
    # Extracting the text
    page = pdf_reader.pages[0]
    extracted_text = page.extract_text()
    
    # Printing the extracted text
    ipca = extracted_text.split('IPCA')[1].split()[2:5] + extracted_text.split('IPCA')[1].split(')')[2].split()[4:6]

    selic = extracted_text.split('Selic')[1].split()[2:5] + extracted_text.split('Selic')[1].split(')')[2].split()[4:6]
    
    pib = extracted_text.split('PIB')[1].split()[6:9] + extracted_text.split('PIB')[1].split(')')[2].split()[4:6]
    
    cambio = extracted_text.split('Câmbio')[1].split()[1:4] + extracted_text.split('Câmbio')[1].split(')')[2].split()[4:6]
    
    ## Compiling into a df
    focus_data = pd.DataFrame({'IPCA': ipca, 'Selic': selic, 'PIB': pib, 'Câmbio': cambio}, index=[f'{curr_year}_4w',
                                                                                                   f'{curr_year}_1w',
                                                                                                   f'{curr_year}_0w',
                                                                                                   f'{next_year}_1w',
                                                                                                   f'{next_year}_0w']).T
# print(extracted_text)

In [ ]:
## Getting the relevant data
focus_data['2024_4w'] = [x.replace('▲', '') for x in focus_data['2024_4w']]
focus_data['2024_1w'] = [x.replace('▲', '') for x in focus_data['2024_1w']]
focus_data['2024_0w'] = [x.replace('▲', '') for x in focus_data['2024_0w']]
focus_data['2025_1w'] = [x.replace('▲', '') for x in focus_data['2025_1w']]
focus_data['2025_0w'] = [x.replace('▲', '') for x in focus_data['2025_0w']]
focus_data['2024_4w'] = [x.replace('▼', '') for x in focus_data['2024_4w']]
focus_data['2024_1w'] = [x.replace('▼', '') for x in focus_data['2024_1w']]
focus_data['2024_0w'] = [x.replace('▼', '') for x in focus_data['2024_0w']]
focus_data['2025_1w'] = [x.replace('▼', '') for x in focus_data['2025_1w']]
focus_data['2025_0w'] = [x.replace('▼', '') for x in focus_data['2025_0w']]
focus_data

# 2. Generating a Image from a Template

In [ ]:
def add_text_to_image(img, text, position, font_text='arial', font_size=29, text_color='blue'):
    
    ## Getting the colors
    colors = {'blue': '#183851', 'gold': '#CA9E67'}
    
    if text_color in colors.keys():
        color = colors[text_color]
    else:
        color = text_colort
    draw = ImageDraw.Draw(img)

    # Setting the font and color for the text
    font = ImageFont.truetype(font_text, font_size)
    draw.text(position, text, fill=color, anchor="mm", font=font)

    return img

In [ ]:
def add_mov(img, idx, ref, position=(540, 960)):
    ## Separating Values
    value_0w = focus_data.at[idx, ref + '_0w']
    value_1w = focus_data.at[idx, ref + '_1w']
    
    # Opening Movement Image
    if value_0w > value_1w:
        mov = Image.open(image_path + r'\up.png')
    elif value_0w < value_1w:
        mov = Image.open(image_path + r'\down.png')
    else:
        mov = Image.open(image_path + r'\equal.png')

    # Resizing the foreground Image
    mov = mov.resize(size=(32, 28))

    # Pasting the foreground image onto the background image
    img.paste(mov, position)

    return img

In [ ]:
## Template path - modify accordingly
image_path = os.path.join(os.path.expanduser('~'), 'Documents')
img = Image.open(image_path + '\Template_Focus.png')

## Parameters
font_date = 'C:/Windows/Fonts/arialbd.ttf'
font_size_date = 40
font_text = 'C:/Windows/Fonts/segoeuil.ttf'
font_size_values = 50
text_color = 'white'

In [ ]:
## Date
months = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
          "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]

date = datetime.today()
date = date.strftime("%d de {} de %Y").format(months[date.month - 1])

img = add_text_to_image(img, date,
                        position=(535, 460),
                        font_text=font_date,
                        font_size=font_size_date,
                        text_color=text_color)

## IPCA
img = add_text_to_image(img, focus_data.at['IPCA', f'{curr_year}_4w'],
                        position=(410, 995),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['IPCA', f'{curr_year}_1w'],
                        position=(585, 995),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['IPCA', f'{curr_year}_0w'],
                        position=(755, 995),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['IPCA', f'{next_year}_0w'],
                        position=(945, 995),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_mov(img, 'IPCA', '2024', position=(817, 985))

img = add_mov(img, 'IPCA', '2025', position=(1002, 985))

## Selic
img = add_text_to_image(img, focus_data.at['Selic', f'{curr_year}_4w'],
                        position=(410, 1138),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['Selic', f'{curr_year}_1w'],
                        position=(585, 1138),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['Selic', f'{curr_year}_0w'],
                        position=(755, 1138),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['Selic', f'{next_year}_0w'],
                        position=(945, 1138),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_mov(img, 'Selic', '2024', position=(817, 1128))

img = add_mov(img, 'Selic', '2025', position=(1002, 1128))

## PIB
img = add_text_to_image(img, focus_data.at['PIB', f'{curr_year}_4w'],
                        position=(410, 1275),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['PIB', f'{curr_year}_1w'],
                        position=(585, 1275),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['PIB', f'{curr_year}_0w'],
                        position=(755, 1275),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['PIB', f'{next_year}_0w'],
                        position=(945, 1275),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_mov(img, 'PIB', '2024', position=(817, 1265))

img = add_mov(img, 'PIB', '2025', position=(1002, 1265))

## Câmbio
img = add_text_to_image(img, focus_data.at['Câmbio', f'{curr_year}_4w'],
                        position=(410, 1412),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['Câmbio', f'{curr_year}_1w'],
                        position=(585, 1412),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['Câmbio', f'{curr_year}_0w'],
                        position=(755, 1412),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_text_to_image(img, focus_data.at['Câmbio', f'{next_year}_0w'],
                        position=(945, 1412),
                        font_text=font_text,
                        font_size=font_size_values,
                        text_color=text_color)

img = add_mov(img, 'Câmbio', '2024', position=(817, 1402))

img = add_mov(img, 'Câmbio', '2025', position=(1002, 1402))

In [ ]:
img = img.convert("RGB")
img.save(os.path.join(os.path.expanduser('~'), 'Documents') + r'\focus_' + last_friday + '.png')
img